In [95]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random 

# Paintings

## Get data

### Import data in your notebook

In [2]:
from vincentvanbot.data import get_data_locally
from sklearn.pipeline import Pipeline
from vincentvanbot.preprocessing.imagevectorizer import ImageVectorizer
from vincentvanbot.preprocessing.imageresizer import ImageResizer
import cv2
from vincentvanbot.preprocessing.utils import get_jpg_link
from vincentvanbot.preprocessing.pipeline import build_pipe

In [29]:
df = get_data_locally()
pipe = build_pipe(dim=(420,360))
df_transformed = pipe.fit_transform(df)
df_transformed.shape

(10, 12)

In [42]:
df_transformed.head()

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME,IMAGE
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/art/a/aachen/adonis.jpg,painting,mythological,German,1601-1650,"[[[109, 106, 87], [116, 113, 94], [111, 108, 8..."
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/art/a/aachen/allegory.jpg,painting,mythological,German,1601-1650,"[[[95, 89, 77], [81, 76, 64], [68, 65, 53], [5..."
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/a/aachen/allegorz.jpg,painting,mythological,German,1601-1650,"[[[125, 104, 59], [94, 72, 37], [63, 39, 16], ..."
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/art/a/aachen/antiope.jpg,painting,mythological,German,1601-1650,"[[[56, 49, 43], [49, 42, 36], [63, 56, 50], [7..."
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/art/a/aachen/athena.jpg,painting,mythological,German,1601-1650,"[[[98, 86, 65], [82, 71, 63], [66, 55, 57], [4..."


### Built KNN

In [54]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [117]:
#print original shape of an image
df_transformed['IMAGE'].iloc[.shape

AttributeError: '_iLocIndexer' object has no attribute 'shape'

In [102]:
df_transformed['IMAGE'].iloc[:10].reshape((10, -1))

AttributeError: 'Series' object has no attribute 'reshape'

In [71]:
# numpy.reshape
X = df_transformed['IMAGE'].iloc[1].reshape((10, -1))
X

array([[ 95,  89,  77, ...,  56,  39,  19],
       [ 77,  86,  67, ...,  99,  76,  45],
       [ 75,  88,  73, ...,  96,  72,  44],
       ...,
       [ 72,  57,  28, ...,  37,  46,  43],
       [ 79,  62,  42, ..., 150, 133, 103],
       [116, 110,  55, ..., 125, 102,  66]], dtype=uint8)

In [ ]:
#df_transformed.images is a numpy array with 10 numpy arrays 360X420 (feature vectors) representing digits
#df_transformed.target ???

In [75]:
model = NearestNeighbors(n_neighbors=2)
model.fit(X)
NearestNeighbors(n_neighbors=2)

A = model.kneighbors_graph(X)
A.toarray()

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.]])

In [132]:
from scipy.spatial import distance
from igraph import *


graph = Graph(directed=True)
graph.add_vertices(len(df_transformed['IMAGE']))

for i in range(len(df_transformed['IMAGE'])):
    distances = [distance.cosine(X[i], feat) for feat in X]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:6] #sort and find most similar items, return the indexes 5 most similar images
    print(idx_closest)
    for j in idx_closest:
        graph.add_edge(i, j, weight=distances[j])
    
summary(graph)

[7, 8, 6, 5, 9]
[7, 5, 8, 6, 0]
[7, 6, 5, 8, 0]
[6, 7, 0, 8, 5]
[8, 6, 7, 5, 0]
[8, 6, 7, 9, 0]
[7, 5, 8, 0, 9]
[8, 6, 0, 5, 9]
[7, 5, 6, 0, 9]
[5, 6, 8, 7, 0]
IGRAPH D-W- 10 50 -- 
+ attr: weight (e)


In [ ]:
def load_image():
    width = random_img_arr.shape[1]
    height = 440
    dim = (width, height)

    # resize image
    resized = cv2.resize(random_img_arr, dim, interpolation = cv2.INTER_AREA)

    print('Resized Dimensions : ',resized.shape)

    plt.imshow(resized)

In [ ]:
img, x = load_image("101_ObjectCategories/airplanes/image_0011.jpg")
print("shape of x: ", x.shape)
print("data type: ", x.dtype)
plt.imshow(img)

In [ ]:
# grab a random image
image_idx = int(len(df_transformed['IMAGE']) * random.random())

# display  image
img = df_transformed['IMAGE'].load_img(df_transformed['IMAGE'][image_idx])
plt.imshow(img)

In [ ]:
# load all the similarity results as thumbnails of height 100
thumbs = []
for idx in idx_closest:
    img = image.load_img(images[idx])
    img = img.resize((int(img.width * 100 / img.height), 100))
    thumbs.append(img)

# concatenate the images into a single image
concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)

# show the image
plt.figure(figsize = (16,12))
plt.imshow(concat_image)

AttributeError: 'Series' object has no attribute 'load_img'

In [103]:
idx1 = int(len(df_transformed['IMAGE']) * random.random())

# run get_shortest_paths
path = graph.get_shortest_paths(idx1, mode=OUT, output='vpath', weights='weight')[0]

# retrieve the images and display them
#results_image = 
plt.figure(figsize = (16,12))
#imshow(results_image)

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

In [109]:
idx2 = df_transformed['IMAGE'].iloc[1].reshape((10, -1))

model.kneighbors(idx2,n_neighbors=2) # Return distances and index of the 2 closest points

(array([[    0.        ,  9971.18147463],
        [    0.        ,  9971.18147463],
        [    0.        , 11970.15175342],
        [    0.        , 11685.80326721],
        [    0.        , 11685.80326721],
        [    0.        , 12798.65739052],
        [    0.        , 10701.71224618],
        [    0.        , 11238.36149089],
        [    0.        , 10581.44115893],
        [    0.        , 10581.44115893]]),
 array([[0, 1],
        [1, 0],
        [2, 3],
        [3, 4],
        [4, 3],
        [5, 8],
        [6, 9],
        [7, 6],
        [8, 9],
        [9, 8]]))

In [ ]:
avg_scores = []  # average score for different k

max_k = 10
cv = 10

# loop over different values of k
for k in range(1, max_k):
  # create knn classifier with k = k
  knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)

  # cross-validate knn on training sample with no_folds
  scores = cross_val_score(knn, data_train, label_train,
                           cv=cv, scoring='accuracy')

  # add avg accuracy to score table
  avg_scores.append(scores.mean())